# Exploring Foursquare
* How much data can I get in a day?
* What categories should I examine and how?
* Will "shrinking box" exploration work?
* Does Folium have a square marker (try in other notebook?)

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [0]:
# define function that extracts the category of the venue
# used in Foursquare section

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [0]:
# @hidden cell
# Foursquare information

CLIENT_ID = 'CWCPNAVXDH3TI1BGS4VED4ANSUKEFGHBA4511GRPYPKPNJRD' # your Foursquare ID
CLIENT_SECRET = 'ALMFEYIENSPH3RV3TQB1NGWTKTANVJ5QTHAVZ5B1GRJWIP21' # your Foursquare Secret
VERSION = '20190427' # Foursquare API version

In [0]:
# Location for sample query - basically my neighborhood in Duluth


super_h = (33.969303, -84.143626)

# corners for Foursquare bounding box search
fsw = (33.960751, -84.139453)  
fne = (33.977991, -84.115742)

# fsw = (33.96, -84.0)  
# fne = (33.97, -84.1)

albany_ne = (32.114878, -83.489320)
albany_sw = (31.517953, -84.222243)

# Foursquare categories
asian = '4bf58dd8d48988d142941735'
# Korean restaurant (sub category of above) 4bf58dd8d48988d113941735
# Indian restaurant (not sub category of above) 4bf58dd8d48988d10f941735

In [0]:
# Define search url and test.
# I took one postal code from the database and defined the search url as follows:

# fsw, fne = albany_sw, albany_ne

LIMIT = 100

url = f'https://api.foursquare.com/v2/venues/explore?&client_id={CLIENT_ID}' + \
      f'&client_secret={CLIENT_SECRET}&v={VERSION}&sw={fsw[0]},{fsw[1]}' + \
      f'&ne={fne[0]},{fne[1]}' + \
      f'&categoryId={asian}&limit={LIMIT}'

url    

results = requests.get(url).json()

In [43]:
# Now convert to dataframe and examine. 
# Note that instead of latitude & longitude, I'm interested in the 'postalCode' field
# I also add 'city' just to be sure I'm getting results from the area that I expect.

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.postalCode','venue.location.lat','venue.location.lng', 'venue.location.state']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

print(nearby_venues.shape)
nearby_venues.sort_values(by=['categories']).tail(50)

(42, 6)


,name,categories,postalCode,lat,lng,state
2,Chicken & Beer (aka BBQ Chicken),Asian Restaurant,30096,33.966146,-84.133552,GA
6,Choo Choo,Asian Restaurant,30096,33.963257,-84.131530,GA
7,Dan Sushi,Asian Restaurant,30096,33.964289,-84.125619,GA
8,Pop Pot 个人食膳锅,Asian Restaurant,30096,33.960809,-84.134458,GA
9,명동본가,Asian Restaurant,30096,33.965920,-84.133805,GA
11,89 Hot Pot,Asian Restaurant,30096,33.961581,-84.135061,GA
14,Boiled Skewer,Chinese Restaurant,30096,33.960873,-84.134070,GA
19,Humble House 寒舍,Chinese Restaurant,30096,33.963557,-84.135183,GA
18,Pings Place In Great Wall Supermarket,Chinese Restaurant,30096,33.963189,-84.135019,GA
17,Best BBQ,Chinese Restaurant,30096,33.963573,-84.135169,GA


In [0]:
print(venues)

[{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4b2d1a43f964a52056ce24e3', 'name': 'Saigon Cafe Duluth', 'location': {'address': '3675 Satellite Blvd #750', 'crossStreet': 'and Pleasant Hill Rd', 'lat': 33.96012236013812, 'lng': -84.13615842313993, 'labeledLatLngs': [{'label': 'display', 'lat': 33.96012236013812, 'lng': -84.13615842313993}], 'postalCode': '30096', 'cc': 'US', 'city': 'Duluth', 'state': 'GA', 'country': 'United States', 'formattedAddress': ['3675 Satellite Blvd #750 (and Pleasant Hill Rd)', 'Duluth, GA 30096', 'United States']}, 'categories': [{'id': '4bf58dd8d48988d142941735', 'name': 'Asian Restaurant', 'pluralName': 'Asian Restaurants', 'shortName': 'Asian', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/asian_', 'suffix': '.png'}, 'primary': True}], 'photos': {'count': 0, 'groups': []}}, 'referralId': 'e-0-4b2d1a43f964a52056ce24e3-0'}, {'reasons':